In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install accelerate -U
# after installation, please restart runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.2 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
rawdatasets = load_dataset("klue","ner")

Generating train split:   0%|          | 0/21008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/ner/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
ner_feature = rawdatasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['B-DT', 'I-DT', 'B-LC', 'I-LC', 'B-OG', 'I-OG', 'B-PS', 'I-PS', 'B-QT', 'I-QT', 'B-TI', 'I-TI', 'O'], id=None), length=-1, id=None)

In [ ]:
# label_names are used below
label_names = ner_feature.feature.names
print(label_names)

['B-DT', 'I-DT', 'B-LC', 'I-LC', 'B-OG', 'I-OG', 'B-PS', 'I-PS', 'B-QT', 'I-QT', 'B-TI', 'I-TI', 'O']


In [ ]:
# Tokens in the datasets are in terms of characters. This causes a problem. They are therefore regenerated in terms of words.
# The function below is used for that purpose
def get_new_words_labels(words, labels):
    new_words = []
    new_labels = []
    prev_label = None
    for i,label in enumerate(labels):
        if prev_label is None:
            prev_label = label
            word = words[i]
        elif words[i] == ' ':
            # new_words.append(word)
            if prev_label % 2 == 1:
                prev_label -= 1
            new_labels.append(prev_label)
            new_words.append(word)
            word=''
            prev_label = label
        elif label % 2 == 1:
            word += words[i]
            prev_label = label
        elif label == prev_label:
            word += words[i]
        elif label == 12 and label != prev_label:
            if prev_label % 2 == 1:
                prev_label -= 1
            new_labels.append(prev_label)
            new_words.append(word)
            word= words[i]
            prev_label = label
        elif label % 2 == 0:
            # new_words.append(word)
            word = words[i]
            # new_labels.append(label)
            prev_label = label

    if len(word) > 0:
        new_words.append(word)
        new_labels.append(label)
    return new_words,new_labels

In [ ]:
# batch version of the get_new_words_labels defined above
def get_new_words_labels_batch(examples):
    all_new_labels = []
    all_new_words = []
    for words,labels in zip(examples["tokens"],examples["ner_tags"]):
        new_words, new_labels = get_new_words_labels(words, labels)
        all_new_words.append(new_words)
        all_new_labels.append(new_labels)
    return all_new_words,all_new_labels

In [ ]:
# Labels corresponding to the new tokens by words are regenerated
def align_labels_with_tokens(labels,word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            if label % 2 == 0 and label != len(label_names)-1:
                label += 1
            new_labels.append(label)
    return new_labels

In [ ]:
from transformers import AutoTokenizer
checkpoint="klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_and_align_labels(examples):
    new_words, all_new_labels = get_new_words_labels_batch(examples)
    tokenized_inputs = tokenizer(new_words,truncation=True,is_split_into_words=True)
    new_labels = []
    for i,labels in enumerate(all_new_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels,word_ids))
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = rawdatasets.map(tokenize_and_align_labels, batched=True,remove_columns=rawdatasets["train"].column_names)

Map:   0%|          | 0/21008 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=48fe2a6ea0be9381fb10ac6df750a64b5a3ca0285a6375ba49cb919a35f58d97
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "klue_ner_roberta_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Cloning https://huggingface.co/chunwoolee0/klue_ner_roberta_model into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.19k/420M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.87k/3.87k [00:00<?, ?B/s]

Download file runs/Jul19_15-24-29_359785df90f9/events.out.tfevents.1689780284.359785df90f9.712.0: 100%|#######…

Clean file training_args.bin:  26%|##5       | 1.00k/3.87k [00:00<?, ?B/s]

Clean file runs/Jul19_15-24-29_359785df90f9/events.out.tfevents.1689780284.359785df90f9.712.0:  12%|#1        …

Clean file pytorch_model.bin:   0%|          | 1.00k/420M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.044900,0.060056,0.936062,0.917584,0.926731,0.983049
2,0.026200,0.046887,0.948417,0.950972,0.949693,0.987351
3,0.014400,0.048677,0.954599,0.955717,0.955157,0.988370


TrainOutput(global_step=7878, training_loss=0.038652972782829385, metrics={'train_runtime': 1076.2787, 'train_samples_per_second': 58.557, 'train_steps_per_second': 7.32, 'total_flos': 1695241464118368.0, 'train_loss': 0.038652972782829385, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/420M [00:00<?, ?B/s]

Upload file runs/Jul24_13-38-40_5bd91c262d74/events.out.tfevents.1690206100.5bd91c262d74.810.0:   0%|         …

To https://huggingface.co/chunwoolee0/klue_ner_roberta_model
   20b37a7..9006e4d  main -> main

   20b37a7..9006e4d  main -> main

To https://huggingface.co/chunwoolee0/klue_ner_roberta_model
   9006e4d..52e1da2  main -> main

   9006e4d..52e1da2  main -> main



'https://huggingface.co/chunwoolee0/klue_ner_roberta_model/commit/9006e4d1e5e3d25806d580c3439630c3faeed20c'

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "chunwoolee0/klue_ner_roberta_model"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("그 제품 삼성 건가요?")

[{'entity_group': 'OG',
  'score': 0.99406624,
  'word': '삼성',
  'start': 5,
  'end': 7}]

In [ ]:
token_classifier("한국, 11년만에 유엔 안보리 비상임이사국 재진입")

[{'entity_group': 'LC',
  'score': 0.71934044,
  'word': '한국',
  'start': 0,
  'end': 2},
 {'entity_group': 'DT',
  'score': 0.99358654,
  'word': '11년',
  'start': 4,
  'end': 7},
 {'entity_group': 'OG',
  'score': 0.9899345,
  'word': '유엔',
  'start': 10,
  'end': 12},
 {'entity_group': 'OG',
  'score': 0.93576455,
  'word': '안보리',
  'start': 13,
  'end': 16}]

In [ ]:
token_classifier("유엔 안보리는 국제 평화 및 안전 유지에 대한 일차적 책임을 지며, 회원국에 대해 국제법적 구속력을 갖는 유일한 핵심 기관이다. ")

[{'entity_group': 'OG',
  'score': 0.9878321,
  'word': '유엔',
  'start': 0,
  'end': 2}]

In [ ]:
token_classifier("류더인 대만 TSMC 회장도 현지 콘퍼런스에서 “받아들일 수 없는 일부 조건이 있다”고 언급한 바 있다.")

[{'entity_group': 'PS',
  'score': 0.99813867,
  'word': '류더인',
  'start': 0,
  'end': 3},
 {'entity_group': 'OG',
  'score': 0.7399861,
  'word': '대만',
  'start': 4,
  'end': 6},
 {'entity_group': 'OG',
  'score': 0.9961631,
  'word': 'TSMC',
  'start': 7,
  'end': 11}]